# Maximum Entropy in Natural Language Processing

When faced with natural language processing tasks such as translation or prediction, the goal is often to find a model that is the most uniform while still fulfilling preset constraints based on prior knowledge/data. By considering models that are uniformly distributed outside of constraints, the hope is to most accurately describe stochastic processes present in data sets by picking the most "general" model. 

For example, if we know the word **water** is always followed the word **bottle**, we can set that tuple as a constraint and have all other words in our data set be equally likely to be followed by any other word.

This maximum entropy function with respect to a conditional probability $p(y|x)$ is given by

$$ H(p) = -\sum_{x,y} \tilde{p}(x)p(y|x)\log p(y|x) $$

However, maximizing $H(p)$ also has the constraints 

$$ p(f_i) = \tilde{p}(f_i) $$

where $f$ is an indicator function with respect to a tuple $(x,y)$, $p(f_i)$ is the expected value of $f$ with respect to the empirical distribution of a tuple $(x,y)$, and $\tilde{p}(f_i)$ is the expected value of $f$ with respect to $p(y|x)$ . We will discuss these and similar functions and their definitions in further detail when implementing this method.

To avoid dealing with these constraints when finding the optimum, we instead minimize the dual of $H(p)$. This dual function is given by

$$ \psi(\lambda) = - \sum_{x} \tilde{p}(x) \log{(\sum_{y} \exp{(\sum_{i} \lambda_i f_i (x,y))})} + \sum_{i} \lambda_i \tilde{p}(f_i)$$

Because our primal function $H(p)$ is symmetric, our dual is therefore unbounded, that is $\lambda \in \rm I\!R$. This dual function should be relatively easier to minimize compared to our original primal problem. 

After calculating our dual variables $\lambda$ we aim to calculate 

$$ p_\lambda (y|x) = \dfrac{\exp{(\sum_{i}\lambda_i f_i(x,y))}}{\sum_{y} \exp{(\sum_{i} \lambda_i f_i (x,y))}} $$

which will give the prediction $y$ given a word $x$ based on the model.

## Function Definitions

To start, we import the NLTK package, which contains useful functions as well as formatted text files.

In [35]:
import nltk
import numpy as np
import scipy.optimize 
from __future__ import division

%load_ext autotime

# only download once
#nltk.download()

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 9 ms


In [ ]:
#from nltk.book import *

Next we define intermediate functions that will combine to form the dual function.

$\tilde{p}(x)$ is the empirical distribution of a word $x$, and so we count the number of occurences of $x$ and divide it by the length of the dataset.

In [36]:
## p~(x): count occurences of word x
def ptildex(text, x):
    return(text.count(x) / len(text))

time: 5 ms


$f_i(x,y)$ is an indicator (and in our case, a feature) function corresponding to a tuple of words (x,y). We will have it return $1$ if the input **tup** matches a specific tuple **z** we chose as a feature. 

In [37]:
def f(tup, z):
    if tup == z: return(1)
    else: return(0)     

time: 5 ms


$\tilde{p}(f)$, as mentioned earlier, is the expected value of a feature function $f(x,y)$ with respect to $\tilde{p}(x,y)$, the empirical distribution of $(x,y)$. Specifically, it is defined as 

$$ \tilde{p}(f) = \sum_{x,y}\tilde{p}(x,y) f(x,y) $$

In [38]:
## p~(x,y): count occurences of a pair (x, y)
def ptildexy(text, tup):
    bg = list(nltk.bigrams(text))
    return(bg.count(tup) / len(bg))

def ptildef(text, bigramset, feat):
    val = float(0)
    for pair in bigramset:
        val += ptildexy(text, pair)*float(f(pair, feat))
    return(val)

time: 7 ms


It would also be convenient to define 
$$\log{(\sum_{y} \exp{(\sum_{i} \lambda_i f_i (x,y))})}$$

In [39]:
def expterm(x, y, lambdas, features):
    total = 0
    for lam, feat in zip(lambdas, features):
        total += lam * f((x,y), feat)
    return(np.exp(total))

def logsum(x, lambdas, features):
    outersum = 0
    for y in sety:
        outersum += expterm(x, y, lambdas, features)
    return(np.log(outersum))

time: 9 ms


Finally, an important NLTK function we will be using is the bigrams function, which accepts a text file input and returns a list of all sequential word pairs. For example, bigrams(yes, no, maybe) will return (yes, no), (no, maybe).

In [40]:
txt = ['hey', 'no', 'ucla', 'math', 'derivative', 'bus', 'computer', 'hello']
bg = list(nltk.bigrams(txt))
print(txt)
print(bg)

['hey', 'no', 'ucla', 'math', 'derivative', 'bus', 'computer', 'hello']
[('hey', 'no'), ('no', 'ucla'), ('ucla', 'math'), ('math', 'derivative'), ('derivative', 'bus'), ('bus', 'computer'), ('computer', 'hello')]
time: 7 ms


Now we can piece together the dual function. It takes a vector of initial lambda values and sums over all $x$ and $y$, which are the sets of unique words in our text file. In our case x and y are equal, as every word can either be predicted or predicted from. 

An important thing to note is that this implementation returns a function value of the opposite sign, as we aim to maximize the dual(and therefore minimize the negative of the dual). 

In [41]:
def negativedual(lambdas):
    firstsum = float(0)
    for x in setx:
        firstsum -= ptildex(text, x) * logsum(x, lambdas, features)
    secondsum = float(0)
    for lam, feat in zip(lambdas, features):
        secondsum += lam * ptildef(text, bigramset, feat)
    return ((-1)*(firstsum + secondsum))

time: 8 ms


To use a Quasi-Newton method we need to also provide a gradient, which we can achieve using simple calculus
$$ \dfrac{\partial \psi(\lambda)}{\partial{\lambda_i}} = -\sum_{x}\tilde{p}(x) \frac{\sum_{y}(\exp{(\sum_{i} \lambda_i f_i(x,y))}f_i (x,y))}{\sum_{y} \exp{(\sum_{i} \lambda_i f_i (x,y))}} + \tilde{p}(f_i)$$

since 

$$ \dfrac{\partial}{\partial \lambda_i} \sum_{i} \lambda_i f_i(x,y) = f_i(x,y) $$

and where $ \dfrac{\partial \psi(\lambda)}{\partial{\lambda_i}}$ is the ith component of our gradient vector.

In [42]:
def negativegrad(lambdas):
    firstsum = 0
    numerator = 0
    denominator = 0
    gradient = np.empty((1,1))
    for i in np.arange(0, len(lambdas)):
        for x in setx:
            for y in sety:
                denominator += expterm(x, y, lambdas, features)
                numerator += denominator * f((x,y), features[i])
            firstsum -= (numerator/denominator) * ptildex(text, x)
        firstsum += ptildef(text, bigramset, features[i])
        gradient = np.append(gradient, -1*firstsum)
    gradient = gradient[1:]
    return(gradient)

time: 8 ms


To find the optimal model given an array of text and features, we initialize $\lambda$ as a vector of values and use the minimize function in the scipy.optimize library. $\lambda$ has an equal number of entries as there are features, as the number of dual variables matches the number of constraints in our primal problem.

## Super Simple Example

Let's start with a simple example of just 4 words and 2 features

In [43]:
text = ['optimization', 'is', 'very', 'complex']
bigramset = list(nltk.bigrams(text))
setx = set(text)
sety = set(text)
features = [('optimization', 'is'), ('very', 'complex')]
lambdas = [0, 0]

result = scipy.optimize.minimize(negativedual, lambdas, method='BFGS', jac=negativegrad)
print(result)

      fun: 1.3862943611198906
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([0.03125   , 0.88764881])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 107
      nit: 0
     njev: 95
   status: 2
  success: False
        x: array([0., 0.])
time: 52 ms


Unfortunately, the BFGS algorithm fails even in this simple case. It is unclear why, but the output indicates that a more optimal value was not able to be found. This is obviously contradictory to what we'd expect, as the gradient is nonzero at our initial point. A reason for this issue could be that the direction the algorithm is attempting to perturb in is very tiny, and the round off error causes it to assume there are no directions that lead to a smaller function value.

We can also attempt to optimize using the Nelder-Mead method, which does not require the gradient.

In [44]:
result = scipy.optimize.minimize(negativedual, lambdas, method='nelder-mead')
print(result)

 final_simplex: (array([[707.88495367, 709.78270615],
       [707.88488304, 709.78263525],
       [707.88487116, 709.78262333]]), array([-117.44582447, -117.44581268, -117.44581069]))
           fun: -117.44582447101908
       message: 'Optimization terminated successfully.'
          nfev: 174
           nit: 91
        status: 0
       success: True
             x: array([707.88495367, 709.78270615])
time: 36 ms


C:\Users\Edwin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """


It worked! The function output indicates that the algorithm finished 94 interations before converging to our minimum. The dual variables are given by the returned vector x, and we can use these to make predictions using $p_\lambda (y|x)$. First we need to implement it

In [45]:
def ygivenx(x, y, lambdas, features):
    numerator = expterm(x, y, lambdas, features)
    denominator = 0
    for yi in sety:
        denominator += expterm(x, yi, lambdas, features)
    return(numerator/denominator)

time: 6 ms


Taking the solution in our optimization results, we plug it into $p_\lambda (y|x)$ along with words $x$ and $y$. The model will then return a value between 0 and 1, which represents the probability of predicting $y$ given the input $x$.

In [46]:
xs = set(text)
ys = set(text)

for x in xs:
    for y in ys:
        print x, "->", y, ": ", np.round(ygivenx(x, y, result.x, features), decimals=3)

very -> very :  0.0
very -> is :  0.0
very -> optimization :  0.0
very -> complex :  1.0
is -> very :  0.25
is -> is :  0.25
is -> optimization :  0.25
is -> complex :  0.25
optimization -> very :  0.0
optimization -> is :  1.0
optimization -> optimization :  0.0
optimization -> complex :  0.0
complex -> very :  0.25
complex -> is :  0.25
complex -> optimization :  0.25
complex -> complex :  0.25
time: 27 ms


Looking at the results of our model, it is (thankfully) consistent with what we expect from it. 

Because our two features required 'is' to be the prediction for 'optimization' and 'complex' to be the prediction for 'very', the model ensured these predictions will always happen(as evident by the probability of 1). All other predictions in these cases are set to 0, as there should be no other options.

In cases where the model was given a word that was not part of a feature, in this case the words 'is' and 'complex', all predictions are uniform, which is consistent with the maximum entropy nature of our model. 

## Harder Example

As our text lengths will get too long to show the probabilities for all permutations, we can write a function that only reports the maximum probability and word associated with it, which in our case would be the prediction. 

In [47]:
def maxprob(x, sety, dualsol, features):
    maxval = 0
    prediction = ""
    for y in sety:
        #print(ygivenx(x, y, dualsol, features))
        if ygivenx(x, y, dualsol, features) > maxval:
            maxval = ygivenx(x, y, dualsol, features)
            prediction = y
    print x, "->", y

time: 7 ms


In [48]:
text = ['In', 'optimization', 'quasi-Newton', 'methods', 'are', 'algorithms', 'for', 'finding', 'local', 
        'maxima', 'and', 'minima', 'of', 'functions', 'In', 'quasi-Newton', 'methods', 'the', 'Hessian',
        'matrix', 'does', 'not', 'need', 'to', 'be', 'computed', 'The', 'first', 'quasi-Newton', 'method',
        'was', 'proposed', 'by', 'William', 'C', 'Davidon', 'a', 'physicist', 'working', 'at', 'Argonne', 
        'National', 'Laboratory', 'He', 'developed', 'the', 'first', 'quasi-Newton', 'algorithm', 'in', '1959']
bigramset = list(nltk.bigrams(text))
setx = set(text)
sety = set(text)
features = [('quasi-Newton', 'methods'), ('local', 'maxima')]
lambdas = [0, 0]

result = scipy.optimize.minimize(negativedual, lambdas, method='BFGS')
print(result)

C:\Users\Edwin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
C:\Users\Edwin\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]


      fun: 3.3552011206846117
 hess_inv: array([[46373.96592142, 12839.59174587],
       [12839.59174587,  3556.02393698]])
      jac: array([-0.00156865, -0.00046   ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 102
      nit: 6
     njev: 23
   status: 2
  success: False
        x: array([34.65549051,  9.42457665])
time: 1.01 s


In [49]:
print "quasi-Newton -> methods: ", np.round(ygivenx('quasi-Newton', 'methods', result.x, features), decimals=3)
print "local -> maxima: ", np.round(ygivenx('local', 'maxima', result.x, features), decimals=3)
maxprob('methods', set(text), result.x, features)
maxprob('Quasi-Newton', set(text), result.x, features)

quasi-Newton -> methods:  1.0
local -> maxima:  0.997
methods -> maxima
Quasi-Newton -> maxima
time: 34 ms


Once again, the predictions are consistent with our features. One important thing to note is even though we have a maximum probability function, the uniformity of our model forces the output to be arbitrary if the given word is not part of a feature. In our example, the maxprob function tells us 'maxima' has the highest probability of being predicted, but in reality all words have equal probability. 

Also, our model treats upper case and lower case forms of the same word differently, but this can be dealt with by preprocessing text. 

## Final Example

Now let's test our optimization method and model on an actual text file, which we can download using the NLTK library. We will be using text3, which is the book of Genesis. One important thing to note is that our algorithm doesn't seem to work with punctuation, we will filter those out beforehand. Also note the duration of each algorithm.

In [51]:
text = list()
punc = [',', '.', ':', ';', '?', '!', '\'', '(', ')', ',)', '.)', ';)', '?)']
for word in text:
    if (word not in punc):
        text.append(str(word))
        
text = text[:1000]
bigramset = list(nltk.bigrams(text[:1000]))
setx = set(text[:1000])
sety = set(text[:1000])
features = [('God', 'created'), ('dry', 'land')]
lambdas = [0, 0]
result = scipy.optimize.minimize(negativedual, lambdas, method='nelder-mead')
print(result)

 final_simplex: (array([[0.00e+00, 0.00e+00],
       [6.25e-05, 0.00e+00],
       [0.00e+00, 6.25e-05]]), array([-0., -0., -0.]))
           fun: -0.0
       message: 'Optimization terminated successfully.'
          nfev: 11
           nit: 3
        status: 0
       success: True
             x: array([0., 0.])
time: 12 ms


In [52]:
text = list()
punc = [',', '.', ':', ';', '?', '!', '\'', '(', ')', ',)', '.)', ';)', '?)']
for word in text3:
    if (word not in punc):
        text.append(str(word))
        
text = text[:2000]
bigramset = list(nltk.bigrams(text[:2000]))
setx = set(text[:2000])
sety = set(text[:2000])
features = [('God', 'created'), ('dry', 'land')]
lambdas = [0, 0]
result = scipy.optimize.minimize(negativedual, lambdas, method='nelder-mead')
print(result)

NameError: name 'text3' is not defined

time: 22 ms


We can see that the algorithm took almost 5 mins to finish optimizing when limiting to the first 1000 words, and over 20 mins when increasing it to 2000 words. This is an indication that it might be a $O(n^2)$ operation, or maybe even exponential. I did not include the full length of the text as it is over 40,000 words, and it isn't hard to imagine how long it would take to finish running on a laptop. However, we can use the previous examples as a proof of concept. 

## Possible Gradient Method

Another popular and computationally cheaper optimization method is a gradient method, which iteratively steps in a direction that minimizes the function(usually determined by the gradient). First we set $\lambda = 0$. Then for each $\lambda_i$ we increment it by $\Delta \lambda_i$ so that $\lambda_i = \lambda_i + \Delta \lambda_i$ where $\Delta \lambda_i$ is the solution to 

$$ \sum_{x,y} \tilde{p}(x) p(y|x) f_i(x,y) \exp{(\Delta \lambda_i \sum_{i} f_i(x,y))} = \tilde{p}(f_i)$$

Subtracting $\tilde{p}(f_i)$ from both sides, we can use the fsolve function in scipy to find the root (in this case $\lambda_i$). We then check for convergence of $\lambda_i$ and repeat the increment of $\lambda_i$ if not.

While I did not have the time to learn how to use the fsolve algorithm and fully implement this method, below is an idea of how to implement the above function, find its root, and loop this process for all $\lambda_i$.

First we code the the function of $\Delta \lambda_i$

In [ ]:
def sumf(x, y, features):
    total = 0
    for feat in features:
        total += f((x,y), feat)
    return(total)

# define each individual feature as dfeat before running
def iterate(dlambda):
    bigramset = list(nltk.bigrams(text))
    totalsum = 0
    for x in setx:
        for y in sety:
            totalsum += ptildex(text, x)*ygivenx(x, y, lambdas, features)*np.exp(dlambda*sumf(x, y, features)) - ptildef(text, bigramset, dfeat)
    return(totalsum)

dfeat = features[0]
scipy.optimize.fsolve(iterate, 0)

Next we implement a loop to increment each $\lambda_i$ and checking for convergence. We can solve each iterative step by using a root finding function.

In [ ]:
lambdas = [0, 0]
for i in np.arange(0, len(lambdas)):
    lambdai = scipy.optimize.fsolve(iterate, lambdas[i])

Using the dual solution to fit the model is the same as our first method. We expect that this optimization method arrives at the same solution as the dual function is concave and therefore only has one global maximum. 

## Summary and Conclusions

We managed to implement and fit a maximum entropy model from scratch! Well, we used libraries like numpy for data structure and scipy for the algorithms, but mostly from scratch. We demonstrated that the maximum entropy model strictly adheres to the features we set for it and randomly predicts when the features are not relevant. Our implementation was very crude and as basic as can be, but it's a good starting off point for more complicated models down the line.

Things we learned along the way were that quasi-Newton methods didn't seem to work when optimizing our dual, and although it isn't very clear why, we can guess that it has to do with the Hessian approximation not allowing the algorithm to iterate properly. However, a method that did not use gradients like Nelder-Mead managed to optimize the dual and properly fit the model. We also found that the implementation we used was at least an $O(n^2)$ operation, and working with over a 1000 words woudl require a powerful computer or server to complete in a reasonable time frame.

There were also aspects of the implementation that could be improved. For example, I could not figure out how to allow multiple function inputs to work with the scipy optimization algorithms, as they only allowed one function input. To work around this,  variables had to take advantage of a global scope, making for messy and hard to track variable values. This could be further investigated and remedied in the future. There were also issues with very large values when the method calculated function values, although this did not seem to cause any problems. 

Other future work could be to complete and compare the gradient method to our first one. We could also attempt to choose features for our model. The features we used thus far have been only to test that the model fitting is successful, but not necessarily to make meaningful and useful predictions. Choosing key features would entail adding new features to our existing set and comparing the new model to our previous one.